# Analysis of voting age population for the 50th California Congressional District.

  


This jupyter notebook analyzes the voting age for the 50th California Congressional District. The geography of the district is mostly comprised of San Diego County but also includes several census tracts in Riverside County. This notebook produces bar charts to examine:  
* the top ten census tracts with the highest population for all ages
* the top ten census tracts with the highest concentration of voters over the age of 29, and;
* the top ten census tracts with the highest level of voters under the age of 30.

*Data source: U.S. Census Bureau, 2014-2018 American Community Survey 5-Year Estimates; Table S2902, Citizen, voting-age population by selected characteristics.* 

In [41]:
import numpy as np
import pandas as pd
import math
from datascience import *

pd.options.display.float_format = '{:.2f}'.format
# pd.set_option('display.max_columns', 30) #can specify for rows too, and change number to maximum you want

In [42]:
pd.__version__

'0.25.3'

### Calling in the dataset


Calling in the voter dataset. Creating a reference dataframe to view the columns to decide which to keep and drop

In [43]:
# call in the dataset
ref = pd.read_csv('ACSST5Y2018.S2902_data_with_overlays_2020-03-03T174358.csv', delimiter = ',')

# avoid renaming columns, exploring column names.
# truncate the dataset, to two lines. column name and descriptions.
ref = ref.head()

In [44]:
ref = ref.head(2)
ref

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,id,Geographic Area Name,Estimate!!Total!!Citizens 18 years and over,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!A...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...
1,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,9.7,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)


In [45]:
# ref.to_csv(r'G:\My Drive\Data\Voting Age Data Census 2018 5yr\Export\Est_Header_.csv')

In [46]:
#specify datatypes so variables read in properly
variable_types = {'Geo.id': 'str', 'GEO.id2': 'str'}

#specify dtype with variable_types we created above
df = pd.read_csv('ACSST5Y2018.S2902_data_with_overlays_2020-03-03T174358.csv', delimiter = ',', 
                     header=[0], skiprows=[1], dtype=variable_types)

In [47]:
df.sort_values(by=['S2902_C01_001E'], ascending=False).head(3)

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
959,1400000US06073018700,"Census Tract 187, San Diego County, California",29371,1760,(X),(X),24413,1850,83.1,3.5,...,5.3,3.4,12060,1144,94.7,3.4,42126,5240,(X),(X)
760,1400000US06073013310,"Census Tract 133.1, San Diego County, California",19058,1483,(X),(X),4695,1169,24.6,5.1,...,6.8,3.2,17575,1579,93.2,3.2,96520,11908,(X),(X)
878,1400000US06073017030,"Census Tract 170.30, San Diego County, California",14047,828,(X),(X),1752,512,12.5,3.4,...,5.4,3.0,13111,784,94.6,3.0,145195,16715,(X),(X)


In [48]:
# new data frame splitting GEO_ID to column 1. Nulls may create an error
new = df["GEO_ID"].str.split("S", n = 1, expand = True) 

# splitting GEO_ID column to use with lookup table later. 
df["GEOID"]= new[1] 

In [49]:
# produce a list of colums with quotes
cols = list(df.columns.values)

# Moving the newly created GEOID from the last position to the first postion. 
cols = cols[-1:] + cols[:-1]

# Committing the changes to the df dataframe.
df = df[cols]

In [50]:
df.dtypes

GEOID             object
GEO_ID            object
NAME              object
S2902_C01_001E     int64
S2902_C01_001M     int64
                   ...  
S2902_C02_019M    object
S2902_C01_020E    object
S2902_C01_020M    object
S2902_C02_020E    object
S2902_C02_020M    object
Length: 83, dtype: object

In [51]:
df.head(2)

,GEOID,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,06065030101,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)
1,06065030103,1400000US06065030103,"Census Tract 301.03, Riverside County, California",1192,214,(X),(X),325,104,27.3,...,29.9,8.8,835,158,70.1,8.8,45333,12408,(X),(X)


In [52]:
# dropping columns by label
df=df.drop(['GEO_ID'], axis = 1)

##### Calling in a lookup table for the census tracts which define the 50th congressional district. This will be merged with primary dataset and to drop census tracts outside the 50th congression district. 

In [53]:
#specify datatypes so GEOID readS in properly
variable_types = {'GEOID': 'str'}

# call in the dataset lookup table for the 50th district only
lookup = pd.read_csv('G:\My Drive\Data\CA Congress Dist 50\lookup table\CA_Congress_Lookup_50th.csv', delimiter = ',', dtype=variable_types)

# dropping columns by label
lookup=lookup.drop(['FID'], axis = 1)

In [54]:
lookup

,GEOID
0,06065051200
1,06065049600
2,06073020029
3,06073020309
4,06073021205
...,...
154,06073016606
155,06073019002
156,06073016503
157,06073016504


In [55]:
# Joining voter data to the look up table and dropping census tracts outside of the congressional district. 
dist_50 = pd.merge(df, lookup, on='GEOID', how='inner')
dist_50.head(2)

,GEOID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,(X),(X),1343,366,31.0,6.9,...,10.8,5.6,3862,484,89.2,5.6,64321,13079,(X),(X)
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,(X),(X),959,221,24.0,4.4,...,4.0,2.1,3842,344,96.0,2.1,105000,22919,(X),(X)


In [56]:
ref

,GEO_ID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M,...,S2902_C02_018E,S2902_C02_018M,S2902_C01_019E,S2902_C01_019M,S2902_C02_019E,S2902_C02_019M,S2902_C01_020E,S2902_C01_020M,S2902_C02_020E,S2902_C02_020M
0,id,Geographic Area Name,Estimate!!Total!!Citizens 18 years and over,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!A...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...,Estimate!!Total!!Citizens 18 years and over!!P...,Margin of Error!!Total MOE!!Citizens 18 years ...,Estimate!!Percent!!Citizens 18 years and over!...,Margin of Error!!Percent MOE!!Citizens 18 year...
1,1400000US06065030101,"Census Tract 301.01, Riverside County, California",883,148,(X),(X),464,130,52.5,9.7,...,20.0,12.4,706,132,80.0,12.4,58710,6512,(X),(X)


##### Dropping columns but keeping data for: 
* total population
* ages 18 to 29
* ages 29 and over

Dropping all other columns

In [57]:
# keep the first 10 columns and drop the rest. 
dist_50_2 = dist_50.drop(dist_50.iloc[:, 10:82:1], axis=1)

dist_50_2.head(2)

,GEOID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C02_001E,S2902_C02_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,(X),(X),1343,366,31.0,6.9
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,(X),(X),959,221,24.0,4.4


In [58]:
# Dropping columns S2902_C02_001E, S2902_C02_001M. 
dist_50_2 = dist_50_2.drop(dist_50.iloc[:, 4:6:1], axis=1)

dist_50_2.head(2)

,GEOID,NAME,S2902_C01_001E,S2902_C01_001M,S2902_C01_002E,S2902_C01_002M,S2902_C02_002E,S2902_C02_002M
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.0,6.9
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.0,4.4


##### Verifying the merge dropped census tracts outside the district. 

In [59]:
# number of rows and columns in df
total_rows=len(dist_50_2.axes[0])
total_cols=len(dist_50_2.axes[1])
print("Number of Rows: "+str(total_rows))
print("Number of Columns: "+str(total_cols))

Number of Rows: 159
Number of Columns: 8


In [60]:
# make list of column names
cols = list(dist_50_2.columns.values)
cols

['GEOID',
 'NAME',
 'S2902_C01_001E',
 'S2902_C01_001M',
 'S2902_C01_002E',
 'S2902_C01_002M',
 'S2902_C02_002E',
 'S2902_C02_002M']

##### Renaming columns:

In [61]:
# Renameing columns
dist_50_2.rename(columns={ 'S2902_C01_001E':'tot_ovr_18',
                         'S2902_C01_001M': 'tot_ovr_18_moe',
                         'S2902_C01_002E': 'tot_18_29',
                         'S2902_C01_002M': 'tot_18_29_moe',
                         'S2902_C02_002E': 'pct_18_29',
                         'S2902_C02_002M': 'pct_18_29_moe'}, inplace=True)

##### Sorting data

In [62]:
dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe
74,06073018700,"Census Tract 187, San Diego County, California",29371,1760,24413,1850,83.1,3.5
103,06073020027,"Census Tract 200.27, San Diego County, California",10622,688,840,348,7.9,3.0
93,06073020013,"Census Tract 200.13, San Diego County, California",9611,534,1480,386,15.4,3.8
106,06073020103,"Census Tract 201.03, San Diego County, California",7736,459,1563,342,20.2,3.9
71,06073017110,"Census Tract 171.10, San Diego County, California",7595,625,1315,416,17.3,4.8
...,...,...,...,...,...,...,...,...
88,06073019107,"Census Tract 191.07, San Diego County, California",1409,125,185,77,13.1,5.2
83,06073019002,"Census Tract 190.02, San Diego County, California",1295,155,234,112,18.1,7.3
139,06073020710,"Census Tract 207.10, San Diego County, California",1262,79,113,39,9.0,2.7
119,06073020213,"Census Tract 202.13, San Diego County, California",1253,147,464,113,37.0,6.9


In [63]:
# getting data types
dist_50_2.dtypes

GEOID             object
NAME              object
tot_ovr_18         int64
tot_ovr_18_moe     int64
tot_18_29          int64
tot_18_29_moe      int64
pct_18_29         object
pct_18_29_moe     object
dtype: object

#####   changing strings to numeric data types

In [64]:
# changing strings to numeric data types
dist_50_2["pct_18_29"] = pd.to_numeric(dist_50_2["pct_18_29"])
dist_50_2["pct_18_29_moe"] = pd.to_numeric(dist_50_2["pct_18_29_moe"])

#####  Calculating the number of those over 29 and under 30

In [65]:
# Calculating the number of those over 29
dist_50_2['tot_ovr_30'] = dist_50_2['tot_ovr_18'] - dist_50_2['tot_18_29']

# finding the inverse percentage of those over 29 years old.
dist_50_2['pct_ovr_30'] = 100 - dist_50_2['pct_18_29']

dist_50_2

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.00,4.40,3043,76.00
2,06065043220,"Census Tract 432.20, Riverside County, California",2970,300,952,252,32.10,6.40,2018,67.90
3,06065043222,"Census Tract 432.22, Riverside County, California",3591,275,552,147,15.40,3.80,3039,84.60
4,06065043246,"Census Tract 432.46, Riverside County, California",4145,393,767,200,18.50,3.60,3378,81.50
...,...,...,...,...,...,...,...,...,...,...
154,06073021205,"Census Tract 212.05, San Diego County, California",5225,611,1287,426,24.60,6.40,3938,75.40
155,06073021206,"Census Tract 212.06, San Diego County, California",2505,219,286,111,11.40,4.10,2219,88.60
156,06073021302,"Census Tract 213.02, San Diego County, California",6147,555,1643,297,26.70,3.40,4504,73.30
157,06073021303,"Census Tract 213.03, San Diego County, California",6304,637,775,286,12.30,3.90,5529,87.70


In [66]:
dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30
74,06073018700,"Census Tract 187, San Diego County, California",29371,1760,24413,1850,83.10,3.50,4958,16.90
103,06073020027,"Census Tract 200.27, San Diego County, California",10622,688,840,348,7.90,3.00,9782,92.10
93,06073020013,"Census Tract 200.13, San Diego County, California",9611,534,1480,386,15.40,3.80,8131,84.60
106,06073020103,"Census Tract 201.03, San Diego County, California",7736,459,1563,342,20.20,3.90,6173,79.80
71,06073017110,"Census Tract 171.10, San Diego County, California",7595,625,1315,416,17.30,4.80,6280,82.70
...,...,...,...,...,...,...,...,...,...,...
88,06073019107,"Census Tract 191.07, San Diego County, California",1409,125,185,77,13.10,5.20,1224,86.90
83,06073019002,"Census Tract 190.02, San Diego County, California",1295,155,234,112,18.10,7.30,1061,81.90
139,06073020710,"Census Tract 207.10, San Diego County, California",1262,79,113,39,9.00,2.70,1149,91.00
119,06073020213,"Census Tract 202.13, San Diego County, California",1253,147,464,113,37.00,6.90,789,63.00


In [67]:
cols = list(dist_50_2.columns.values)
cols

['GEOID',
 'NAME',
 'tot_ovr_18',
 'tot_ovr_18_moe',
 'tot_18_29',
 'tot_18_29_moe',
 'pct_18_29',
 'pct_18_29_moe',
 'tot_ovr_30',
 'pct_ovr_30']

##### Creating a census tract label column for use in graphs

In [68]:
# new data frame splitting Census tract number to column 2. Nulls may create an error
new = dist_50_2["NAME"].str.split(" ", n = 5, expand = True) 
new = new[2].str.split(",", n = 5, expand = True) 
new.head(2)

,0,1
0,432.16,
1,432.18,


In [69]:
#Creating new column from split for CT. 
dist_50_2["Census Tract"]= new[0]
dist_50_2['Census Tract'] = 'CT ' + dist_50_2['Census Tract'].astype(str)
dist_50_2.head(2)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract
0,06065043216,"Census Tract 432.16, Riverside County, California",4331,437,1343,366,31.00,6.90,2988,69.00,CT 432.16
1,06065043218,"Census Tract 432.18, Riverside County, California",4002,338,959,221,24.00,4.40,3043,76.00,CT 432.18


In [70]:
# new data frame splitting County to column 1. Nulls may create an error
new = dist_50_2["NAME"].str.split(" ", n =3, expand = True) 
new = new[3].str.split(" C", n =1, expand = True)

# Creating new County column. 
dist_50_2["County"]= new[0] 

##### Formatting data

In [71]:
format_dict = { 'tot_ovr_18':'{:,}' , 'tot_ovr_18_moe': '{:,}',
                         'tot_18_29': '{:,}', 'tot_18_29_moe': '{:,}',
                         'pct_18_29' : '{:.1f}%',
                         'pct_18_29_moe' : '{:.1f}%',
                         'pct_ovr_30' : '{:.1f}%',
                         'tot_ovr_30':'{:,}'}

In [72]:
dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False).style.format(format_dict)

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract,County
74,06073018700,"Census Tract 187, San Diego County, California","29,371","1,760","24,413","1,850",83.1%,3.5%,"4,958",16.9%,CT 187,San Diego
103,06073020027,"Census Tract 200.27, San Diego County, California","10,622",688,840,348,7.9%,3.0%,"9,782",92.1%,CT 200.27,San Diego
93,06073020013,"Census Tract 200.13, San Diego County, California","9,611",534,"1,480",386,15.4%,3.8%,"8,131",84.6%,CT 200.13,San Diego
106,06073020103,"Census Tract 201.03, San Diego County, California","7,736",459,"1,563",342,20.2%,3.9%,"6,173",79.8%,CT 201.03,San Diego
71,06073017110,"Census Tract 171.10, San Diego County, California","7,595",625,"1,315",416,17.3%,4.8%,"6,280",82.7%,CT 171.10,San Diego
57,06073016701,"Census Tract 167.01, San Diego County, California","7,373",466,"1,593",433,21.6%,5.1%,"5,780",78.4%,CT 167.01,San Diego
123,06073020306,"Census Tract 203.06, San Diego County, California","7,351",662,"2,489",524,33.9%,5.0%,"4,862",66.1%,CT 203.06,San Diego
76,06073018802,"Census Tract 188.02, San Diego County, California","6,877",450,800,205,11.6%,2.7%,"6,077",88.4%,CT 188.02,San Diego
89,06073019207,"Census Tract 192.07, San Diego County, California","6,746",546,"1,994",395,29.6%,4.9%,"4,752",70.4%,CT 192.07,San Diego
72,06073018611,"Census Tract 186.11, San Diego County, California","6,350",456,"1,400",395,22.0%,5.3%,"4,950",78.0%,CT 186.11,San Diego


#####  Creating new dataframes for:
* top 10 census tracts with the most people
* top 10 census tracts with the most people over 29
* top 10 census tracts with the most people under 30


In [73]:
top_10_ct = dist_50_2.sort_values(by=['tot_ovr_18'], ascending=False)
top_10_ct =  top_10_ct.head(10)

In [74]:
top_10_ct_undr_30 = dist_50_2.sort_values(by=['tot_18_29'], ascending=False)
top_10_ct_undr_30 =  top_10_ct_undr_30.head(10)

In [75]:
top_10_ct_ovr_29 = dist_50_2.sort_values(by=['tot_ovr_30'], ascending=False)
top_10_ct_ovr_29 =  top_10_ct_ovr_29.head(10)
top_10_ct_ovr_29

,GEOID,NAME,tot_ovr_18,tot_ovr_18_moe,tot_18_29,tot_18_29_moe,pct_18_29,pct_18_29_moe,tot_ovr_30,pct_ovr_30,Census Tract,County
103,06073020027,"Census Tract 200.27, San Diego County, California",10622,688,840,348,7.90,3.00,9782,92.10,CT 200.27,San Diego
93,06073020013,"Census Tract 200.13, San Diego County, California",9611,534,1480,386,15.40,3.80,8131,84.60,CT 200.13,San Diego
71,06073017110,"Census Tract 171.10, San Diego County, California",7595,625,1315,416,17.30,4.80,6280,82.70,CT 171.10,San Diego
106,06073020103,"Census Tract 201.03, San Diego County, California",7736,459,1563,342,20.20,3.90,6173,79.80,CT 201.03,San Diego
76,06073018802,"Census Tract 188.02, San Diego County, California",6877,450,800,205,11.60,2.70,6077,88.40,CT 188.02,San Diego
57,06073016701,"Census Tract 167.01, San Diego County, California",7373,466,1593,433,21.60,5.10,5780,78.40,CT 167.01,San Diego
157,06073021303,"Census Tract 213.03, San Diego County, California",6304,637,775,286,12.30,3.90,5529,87.70,CT 213.03,San Diego
59,06073016802,"Census Tract 168.02, San Diego County, California",5963,610,805,259,13.50,3.70,5158,86.50,CT 168.02,San Diego
7,06065043252,"Census Tract 432.52, Riverside County, California",6021,469,967,267,16.10,3.90,5054,83.90,CT 432.52,Riverside
87,06073019106,"Census Tract 191.06, San Diego County, California",5993,513,1011,220,16.90,3.40,4982,83.10,CT 191.06,San Diego


In [76]:
# number of rows and columns in df
total_rows=len(top_10_ct.axes[0])
total_cols=len(top_10_ct.axes[1])
print("Number of Rows: "+str(total_rows))
print("Number of Columns: "+str(total_cols))

Number of Rows: 10
Number of Columns: 12


In [87]:
import plotly.graph_objects as go

x = top_10_ct['Census Tract']
y = top_10_ct['tot_ovr_18']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            texttemplate = "%{y:,}",  # specify text template for formatting number labels
            textposition='auto'
            
        )])



fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)', 
   
})

# Customize aspect
fig.update_traces(marker_color='rgb(158,154,200)', 
                  )
fig.update_layout(title_text='Top 10 Census Tracts with the greatest number of people over 18')

fig.show()




# # Customize aspect
# fig.update_traces(marker_color='rgb(158,202,225)', marker_line_color='rgb(8,48,107)',
#                   marker_line_width=1.5, opacity=0.6)
# fig.update_layout(title_text='January 2013 Sales Report')

# fig.show()



In [86]:
import plotly.graph_objects as go

x = top_10_ct_ovr_29['Census Tract']
y = top_10_ct_ovr_29['tot_ovr_30']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            texttemplate = "%{y:,}",  # specify text template for formatting number labels
            textposition='auto'
            
        )])



fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)', 
   
})

# Customize aspect
fig.update_traces(marker_color='rgb(158,154,200)', 
                  )
fig.update_layout(title_text='Top 10 census tracts with the greatest number of people over the age of 29')

fig.show()




In [79]:
import plotly.graph_objects as go

x = top_10_ct_undr_30 ['Census Tract']
y = top_10_ct_undr_30 ['tot_18_29']

# Use textposition='auto' for direct text
fig = go.Figure(data=[go.Bar(
            x=x, y=y,
            text=y,
            texttemplate = "%{y:,}",  # specify text template for formatting number labels
            textposition='auto', 
            
        )])



fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)', 
   
})

# Customize aspect
fig.update_traces(marker_color='rgb(158,154,200)', 
                  )
fig.update_layout(title_text='Top 10 census tracts with the greatest number of people under the age of 30')

fig.show()